In [39]:
!pip install setfit

In [40]:
import evaluate
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset, DatasetDict
from setfit import SetFitModel, TrainingArguments as SetFitTrainingArguments, Trainer as SetFitTrainer

In [41]:
data = load_dataset("SetFit/SentEval-CR")

Repo card metadata block was not found. Setting CardData to empty.


In [42]:
df = data["train"].to_pandas().sample(100).reset_index(drop=True)

print(df.shape)

(100, 3)


In [43]:
df.head()

,text,label,label_text
0,"i 've had this thing for less than a month , t...",0,negative
1,"once you set it up , it 's great .",1,positive
2,the only thing i do n 't like about this is th...,0,negative
3,now i know why - they are pieces of junk .,0,negative
4,"however , this ultracompact allows the photogr...",1,positive


In [44]:
Xtrain, Xval = train_test_split(df, stratify=df["label_text"], test_size=.3)

print(Xtrain.shape, Xval.shape)

(70, 3) (30, 3)


In [45]:
class_indices = {1:'positive', 0:'negative'}

class_indices

{1: 'positive', 0: 'negative'}

In [46]:
data_dict = DatasetDict({
                        "train":Dataset.from_pandas(Xtrain.reset_index(drop=True)),
                        "val":Dataset.from_pandas(Xval.reset_index(drop=True)),
                        "test":data["test"]
                    }).remove_columns(["label_text"])

data_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 70
    })
    val: Dataset({
        features: ['text', 'label'],
        num_rows: 30
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 753
    })
})

In [47]:
model = SetFitModel.from_pretrained("sentence-transformers/all-mpnet-base-v2",
                                    labels=["positive", "negative"],
                                    use_differentiable_head=True,
                                    head_params={"out_features": 2}).to("cuda:0")

model

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [48]:
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(preds, labels):
    accuracy = accuracy_metric.compute(predictions=preds, references=labels)
    f1 = f1_metric.compute(predictions=preds, references=labels, average="weighted")
    return {
        "accuracy": accuracy["accuracy"],
        "f1": f1["f1"],
    }

In [49]:
training_args = SetFitTrainingArguments(output_dir='model_1',
                                        batch_size=32,
                                        num_epochs=10,
                                        num_iterations=20,
                                        eval_strategy="epoch",
                                        save_strategy="epoch",
                                        report_to="none",
                                        load_best_model_at_end=True)

model_trainer = SetFitTrainer(model=model,
                              args=training_args,
                              train_dataset=data_dict['train'],
                              eval_dataset=data_dict['val'],
                              metric=compute_metrics)

Map:   0%|          | 0/70 [00:00<?, ? examples/s]

In [50]:
model.predict(data_dict["train"][0]["text"])

'positive'

In [51]:
model_trainer.train()

***** Running training *****
  Num unique pairs = 2800
  Batch size = 32
  Num epochs = 10


Epoch,Training Loss,Validation Loss
1,0.241700,0.061100
2,0.000500,0.058074
3,0.000300,0.058261
4,0.000200,0.058320
5,0.000200,0.058657
6,0.000100,0.058426
7,0.000100,0.058453
8,0.000100,0.058699
9,0.000100,0.058679
10,0.000100,0.058719


The `max_length` is `None`. Using the maximum acceptable length according to the current model body: 384.


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3 [00:00<?, ?it/s]

In [54]:
import pandas as pd
pd.DataFrame([model_trainer.evaluate(data_dict["test"])])

***** Running evaluation *****


,accuracy,f1
0,0.903054,0.902355
